# Store and retrieval


In [ ]:
%pip install chromadb==1.0.15
%pip install sentence-transformers==5.0.0
%pip install langchain==0.3.26
%pip install langchain-community==0.3.27
%pip install langchain-chroma==0.2.4
%pip install langchain-huggingface==0.3.0
%pip install langchain-openai==0.3.27

## 1. Load and split the documents

In [ ]:
%load_ext dotenv
%dotenv

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# load the document and split it into chunks
loader = TextLoader("./content/max-und-moritz.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size = 2500, chunk_overlap = 200)
docs = text_splitter.split_documents(documents)

## 2. Store documents into the vector database

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# prepare the embedding model
embedding_function = HuggingFaceEmbeddings(model_name = 'intfloat/multilingual-e5-large')

# load it into Chroma
db = Chroma(embedding_function = embedding_function, persist_directory = './chromadb/')
db.add_documents(docs)

## 3. Query the documents

In [ ]:
query = 'Wie heisst der Lehrer?'
#query = 'Who is the sensei?'
#query = 'Was machen die Hühner unmittelbar vor ihrem Tod?'
#query = 'Wie viele Hühner hatte Witwe Bolte?'
#query = 'Was machen Max und Moritz mit dem Schneider und wie heisst er?'

results = db.similarity_search_with_score(query)

for result in results:
    print(f'Score: {result[1]}')
    print(f'Text:\n{result[0].page_content}\n\n')

## 4. RAG

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

retriever = db.as_retriever()

template = """Beantworte die Frage nur aufgrund der folgenenden Informationen:
{context}

Frage: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model_name = 'gpt-4o'
model = ChatOpenAI(model_name = model_name)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke(query)